In [1]:
import numpy as np
from scipy.optimize import minimize

def parse_board(board):
    return np.array([[int(c) if c.isdigit() else 0 for c in row.strip() if c != " "] for row in board.strip().split('\n')])

def print_board(board):
    for i, row in enumerate(board):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - - -")
        for j, val in enumerate(row):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")
            if j == 8:
                print(val)
            else:
                print(str(val) + " ", end="")
    print()

def board_is_solved(board):

    for num in range(1, 10):
        # Check rows
        for i in range(9):
            count = 0
            for j in range(9): #cycle through columns of a fixed row
                if board[i][j] == num:
                    count += 1
                if count > 1: #if a number appear more than once in a row
                    return False
            if count < 1: #if a number doesn't appear in a row
                return False
                    
        # check columns
        for j in range(9):
            count = 0
            for i in range(9): #cycle through rows of a fixed column
                if board[i][j] == num:
                    count += 1
                if count > 1: #if a number appear more than once in a row
                    return False
            if count < 1: #if a number doesn't appear in a row
                return False

    # Check boxes
        for box_x in range(1, 3):
            for box_y in range(1, 3):
                count = 0
                for i in range(box_y * 3, box_y * 3 + 3):
                    for j in range(box_x * 3, box_x * 3 + 3):
                        if board[i][j] == num:
                            count += 1
                        if(count > 1): #if a number appears more than one time in a box 3x3
                            return False
                if count < 1: #if a number doesn't appear in a box
                    return False
    return True


def board_is_full(board):
    
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                return False
    return True

In [ ]:
def coords_already_full(board):
    
    list = []
    
    for i in range(9):
        for j in range(9):
            if board[i][j] != 0:
                list.append((i,j))

def fitness(board, already_filled):
    fitness_score = 0
    
    for i in range(9):
        for j in range(9):
            if not (i,j) in already_filled:
                
                # check collision in row
                for j2 in range(9):
                    # if the element found (different from the starting element) has equal value it's a collision
                    if board[i, j2] == board[i][j] and (i,j2) != (i,j): 
                        fitness_score += 1
                        
                # check collision in column
                for i2 in range(9):
                    if board[i2, j] == board[i][j] and (i2,j) != (i,j):
                        fitness_score += 1
                        
                #check collision in box 3x3
                box_x = j // 3
                box_y = i // 3
                for i2 in range(box_y*3, box_y*3 + 3):
                    for j2 in range(box_x*3, box_x*3 + 3):
                        if board[i2, j2] == board[i][j] and (i2,j) != (i,j):
                            fitness_score += 1
    
    return fitness_score

### TEST SECTION

In [ ]:
# Example Board
sudoku_board = """
37. 5....6
... 36. .12
... .91 75.
... 154 .7.
..3 .7. 6..
.5. 638 ...
.64 98. ...
59. .26 ...
2.. ..5 .64
"""

board = parse_board(sudoku_board)
print("Example Sudoku:")
print_board(board)


if(solve_board(board)):
    print("Soluzione: ")
    print_board(board)
else:
    print("Board non risolvibile")